In [5]:
import numpy as np
import pandas as pd
import mysql.connector as sql
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim as nm
import folium as fm
from folium.plugins import MarkerCluster
from tqdm.notebook import tqdm

#%matplotlib qt


In [6]:
#IMPORTING ALL THREE DATA FRAMES 
df1 = pd.read_csv('events_data.csv')
server = sql.connect(host='cpanel.insaid.co', database='Capstone1', user='student', password='student')

df2 = pd.read_sql('SELECT * FROM gender_age_train', con=server)
df3 = pd.read_sql('SELECT * FROM phone_brand_device_model', con=server)
server.close

#ADDITIONAL REQUIRED INFO
eng_names = np.array([['华为','Huawei'],
                      ['小米','Xiaomi'],
                      ['三星','Samsung'],
                      ['vivo','vivo'],
                      ['OPPO','OPPO'],
                      ['魅族','Meizu'],
                      ['酷派','Coolpad'],
                      ['乐视','LeEco'],
                      ['联想','Lenovo'],
                      ['HTC','HTC']])

In [7]:
# Some output formatting assistance
def line_break():
    print('-------------------------------------------------------------------------------------')



# Check 1 - Whether longitude and latitude are always missing in pairs:
# This code creates two dataframes, containing the index values (row numbers) of the rows where either one is missing, and then checks if the two dataframes are identical.
# If they are identical, then they are always missing in pairs.
Check1 = df1[df1['latitude'].isnull()].index.equals(df1['longitude'].isnull().index)
print("Check 1 - Latitude and Longitude are always missing in pairs - " + str(Check1))

line_break()

# Check 2 - Whether each city only exists in one state
# This code creates a dataframe containing unique city-state pairs, and the length of this dataframe is compared to the number of unique cities.
# If they are of different length, then we cannot calculate the state using the city
city_state_pairs = df1[df1['state'].notnull()][['city','state']].drop_duplicates()
unique_cities = df1['city'].nunique()
print('Check 2 - There are ' + str(len(city_state_pairs)) + ' unique city-state pairs, and ' + str(unique_cities) + ' unique cities')
#These two numbers are not equal. This means that some cities are present in multiple states. To verify:
city_state_pairs['city'].value_counts()[0:11]

line_break()

# Check 3 - Whether we can use coordinates to find the state
Check3a = df1[df1['latitude'].isnull()].index.tolist()
Check3b = df1[df1['state'].isnull()].index.tolist()
for i in Check3a:
    if i in Check3b:
        print('Check 3 - In row ' + str(i) + ', both the coordinates and the city are missing')
# This prints nothing; therefore, we can use coordinates to find the city.
# This is foolproof, as each coordinate pair MUST refer to only one state

line_break()

# Check 4 - Keep track of number of unique entries in "state"
# After adding the states using geopy, if the syntax is wrong, then this will help detect that:
print('Check 4 - There are ' + str(df1['state'].nunique()) + ' unique entries in the "state" column.')

Check 1 - Latitude and Longitude are always missing in pairs - False
-------------------------------------------------------------------------------------
Check 2 - There are 943 unique city-state pairs, and 933 unique cities
-------------------------------------------------------------------------------------
-------------------------------------------------------------------------------------
Check 4 - There are 32 unique entries in the "state" column.


In [8]:
# This is a list containing the index values of all rows where state is a null value.
null_records = df1[df1['state'].isnull()].index.tolist()
# This section of code fills in those values via geopy and a loop over the rows with null values
geolocator = nm(user_agent="my-applications")
for i in null_records:
    location = geolocator.reverse([df1.at[i,'latitude'],df1.at[i,'longitude']])
    # the code for location is taken from https://www.geeksforgeeks.org/python-remove-spaces-from-a-string/
    df1.at[i,'state'] = location.raw['address'].get('state', '').replace(' ','')

print(df1['state'].nunique())
df1 = df1[df1['state'] == "Maharashtra"]

Check 5 - There are now 32 unique entries in the "state" column.


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677168 entries, 9 to 3252946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677168 non-null  int64  
 1   device_id  677096 non-null  float64
 2   timestamp  677168 non-null  object 
 3   longitude  677105 non-null  float64
 4   latitude   677105 non-null  float64
 5   city       677168 non-null  object 
 6   state      677168 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 41.3+ MB


In [12]:
# Studying the null values in the coordinates. 
# This dataframe contains only the rows with null values in the coordinates
t1 = df1[df1['longitude'].isnull()]
print(t1.info())
line_break()
# This shows the device_ids present in the records with missing coordinates. 
# Note that there are only 3 unique device_ids, with 21 missing coordinate pairs each.
print(t1['device_id'].value_counts())
# This stores the 3 unique device_ids, so we can study them across the whole database.
bad_coords_id = t1['device_id'].value_counts().index

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 20586 to 3240304
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   event_id   63 non-null     int64  
 1   device_id  63 non-null     float64
 2   timestamp  63 non-null     object 
 3   longitude  0 non-null      float64
 4   latitude   0 non-null      float64
 5   city       63 non-null     object 
 6   state      63 non-null     object 
dtypes: float64(3), int64(1), object(3)
memory usage: 3.9+ KB
None
-------------------------------------------------------------------------------------
-7.364841e+18    21
 7.018600e+18    21
 5.115185e+18    21
Name: device_id, dtype: int64


In [13]:
# These count the number of unique coordinate pairs that the 3 device_ids have across the whole database
# Note that each of the three has one unique coordinate pair associated with it across ALL records. This means that the device was used from the same location all the time
# There are well over a thousand rows for each device_id with the coordinates present, so we can safely substitute the missing values with the device_id's coordinates.
print(df1[df1['device_id'] == bad_coords_id[0]][['latitude','longitude']].value_counts())
print(df1[df1['device_id'] == bad_coords_id[0]][['latitude','longitude']].info())
line_break()
print(df1[df1['device_id'] == bad_coords_id[1]][['latitude','longitude']].value_counts())
print(df1[df1['device_id'] == bad_coords_id[1]][['latitude','longitude']].info())
line_break()
print(df1[df1['device_id'] == bad_coords_id[2]][['latitude','longitude']].value_counts())
print(df1[df1['device_id'] == bad_coords_id[2]][['latitude','longitude']].info())

latitude  longitude
18.54074  73.931944    2604
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2625 entries, 1678 to 3251982
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   2604 non-null   float64
 1   longitude  2604 non-null   float64
dtypes: float64(2)
memory usage: 61.5 KB
None
-------------------------------------------------------------------------------------
latitude   longitude
18.575747  73.926223    1894
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1915 entries, 822 to 3252657
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   latitude   1894 non-null   float64
 1   longitude  1894 non-null   float64
dtypes: float64(2)
memory usage: 44.9 KB
None
-------------------------------------------------------------------------------------
latitude   longitude
18.556741  73.88868     1864
dtype: int64
<class

In [14]:
# Doing the substitution here
for i in (range(0,3)):
    values = {}
    replacement = df1[df1['device_id'] == bad_coords_id[i]][['latitude','longitude']].value_counts().index[0]
    values['latitude'] = replacement[0]
    values['longitude'] = replacement[1]
    df1[df1['device_id'] == bad_coords_id[i]] = df1.fillna(value = values)
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677168 entries, 9 to 3252946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677168 non-null  int64  
 1   device_id  677096 non-null  float64
 2   timestamp  677168 non-null  object 
 3   longitude  677168 non-null  float64
 4   latitude   677168 non-null  float64
 5   city       677168 non-null  object 
 6   state      677168 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 57.5+ MB


In [15]:
# This takes out the 9 points outside of Maharashtra. 
# The most eastern point of the 9 was in Kabul at ~69 longitude, so a cutoff of 72 is used here.
df1 = df1[df1['longitude'] > 72]
df1.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677159 entries, 9 to 3252946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677159 non-null  int64  
 1   device_id  677087 non-null  float64
 2   timestamp  677159 non-null  object 
 3   longitude  677159 non-null  float64
 4   latitude   677159 non-null  float64
 5   city       677159 non-null  object 
 6   state      677159 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 41.3+ MB


In [39]:
# EVERYTHING TILL HERE IS FINALIZED
# EVERYTHING AFTER HERE IS STILL BEING WORKED ON
df1.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 677159 entries, 9 to 3252946
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   event_id   677159 non-null  int64  
 1   device_id  677087 non-null  float64
 2   timestamp  677159 non-null  object 
 3   longitude  677159 non-null  float64
 4   latitude   677159 non-null  float64
 5   city       677159 non-null  object 
 6   state      677159 non-null  object 
dtypes: float64(3), int64(1), object(3)
memory usage: 41.3+ MB


In [40]:
t1 = df1.copy()
t1[0:15]

,event_id,device_id,timestamp,longitude,latitude,city,state
9,720049,4.928556e+18,2016-05-06 19:00:52,72.844795,18.974929,Mumbai,Maharashtra
11,13998,3.978026e+18,2016-05-06 13:12:42,73.875196,18.615791,Pune,Maharashtra
20,1378252,-4.384474e+18,2016-05-05 04:20:02,73.914179,18.619081,Pune,Maharashtra
21,185325,-7.369690e+18,2016-05-02 20:07:07,72.874552,19.014537,Mumbai,Maharashtra
22,1861432,3.965811e+18,2016-05-02 20:49:13,73.938826,18.582846,Pune,Maharashtra
25,2923724,5.178608e+18,2016-05-05 21:14:28,72.859311,18.990110,Mumbai,Maharashtra
33,1851307,4.918127e+18,2016-05-03 11:07:32,77.630324,19.990620,Pusad,Maharashtra
34,1695686,-1.844276e+18,2016-05-05 19:06:46,77.354427,21.255712,Anjangaon,Maharashtra
41,1171529,3.653643e+18,2016-05-03 07:57:10,72.838406,19.014749,Mumbai,Maharashtra
42,2523586,-7.818282e+18,2016-05-05 15:33:49,73.929706,18.623336,Pune,Maharashtra


In [48]:
t2 = t1[t1['device_id'].isnull()]
t1 = t1[t1['device_id'].notnull()]


In [68]:
print(t2[['longitude','latitude']].nunique())


longitude    3
latitude     3
dtype: int64
